## Video to Audio Conversion

### Install necessary libraries

In [2]:
# !pip3 install imageio==2.4.1
# !pip install --upgrade imageio-ffmpeg
%pip install moviepy
%pip install -U sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import moviepy.editor as mp
import pandas as pd
import numpy as np

### Video to Audio Conversion

In [4]:
clip = mp.VideoFileClip(r"/home/shivam/Downloads/linked.mp4") 
clip.audio.write_audiofile(r"converted.wav")

MoviePy - Writing audio in converted.wav


MoviePy - Done.


# Audio to text conversion

### Stt using pvleopard

In [5]:
%pip install pvleopard
import pvleopard
leopard = pvleopard.create(access_key='clzWvIxwJZXvEIc4rhS9oBA3yoJ8GzcTqZfvn6KuqL/M7FPcB2HJWQ==')

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
transcript, words = leopard.process_file('converted.wav')
print(transcript)

Good morning everybody my name is Jason coup I'm going to be teaching this class in introduction to algorithms with two other instructors here faculty in the department Eric demaine and Justin Solomon their excellent people and so they will be working on teaching this class with me I will be teaching the first lecture and will have each of them teach one of the next to lectures and then we'll go from there yeah so that's this is intro to algorithms okay so we're gonna start talking about this course content now what is this course about it's about algorithms right introduction algorithms really what the course is about is teaching you to solve computational problems but it's more than that it's not just about teaching you to solve computational problems so solve one so computational problems but it's more than that it's also about communicating those solutions to others and being able to communicate that your way of solving the problem is correct and efficient okay so it's about two mo

# Decreasing the search space

### Employing TF-IDF 

In [7]:
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/shivam/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
vectorizer = TfidfVectorizer(max_features=10,stop_words=stopwords.words('english'))    

In [10]:
corpus = [ transcript]
X = vectorizer.fit_transform(corpus).todense()
df = pd.DataFrame(X, columns=vectorizer.get_feature_names_out())
df

,algorithm,class,kind,like,okay,one,problem,right,time,want
0,0.320202,0.172416,0.172416,0.166259,0.221678,0.21552,0.240151,0.763558,0.21552,0.166259


In [11]:
X1 = np.array(X).flatten()
X2  = vectorizer.get_feature_names_out().flatten()
X2

array(['algorithm', 'class', 'kind', 'like', 'okay', 'one', 'problem',
       'right', 'time', 'want'], dtype=object)

In [12]:
arr = np.array([X1,X2]).T
arr=sorted(arr,key=lambda x:x[0],reverse=True)
arr

[array([0.7635577996483698, 'right'], dtype=object),
 array([0.3202016579170583, 'algorithm'], dtype=object),
 array([0.2401512434377937, 'problem'], dtype=object),
 array([0.22167807086565572, 'okay'], dtype=object),
 array([0.21552034667494308, 'one'], dtype=object),
 array([0.21552034667494308, 'time'], dtype=object),
 array([0.17241627733995446, 'class'], dtype=object),
 array([0.17241627733995446, 'kind'], dtype=object),
 array([0.1662585531492418, 'like'], dtype=object),
 array([0.1662585531492418, 'want'], dtype=object)]

# Making chunks

In [13]:
import os
try :
    os.mkdir('chunks')
except: 
    pass

In [14]:
%pip install pydub

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [17]:
from pydub import AudioSegment
from pydub.utils import make_chunks

myaudio = AudioSegment.from_file("converted.wav" , "wav") 
chunk_length_ms = 100000 # pydub calculates in millisec
chunks = make_chunks(myaudio, chunk_length_ms) #Make chunks of 100 sec

#Export all of the individual chunks as wav files
chunks_list=[]
for i, chunk in enumerate(chunks):
    chunk_name = "./chunks/{0}chunk.wav".format(i)
    print ("exporting", chunk_name)
    chunk.export(chunk_name, format="wav")
    chunks_list.append(chunk_name)

exporting ./chunks/0chunk.wav
exporting ./chunks/1chunk.wav
exporting ./chunks/2chunk.wav
exporting ./chunks/3chunk.wav
exporting ./chunks/4chunk.wav
exporting ./chunks/5chunk.wav
exporting ./chunks/6chunk.wav
exporting ./chunks/7chunk.wav
exporting ./chunks/8chunk.wav
exporting ./chunks/9chunk.wav
exporting ./chunks/10chunk.wav
exporting ./chunks/11chunk.wav
exporting ./chunks/12chunk.wav
exporting ./chunks/13chunk.wav
exporting ./chunks/14chunk.wav
exporting ./chunks/15chunk.wav
exporting ./chunks/16chunk.wav
exporting ./chunks/17chunk.wav
exporting ./chunks/18chunk.wav
exporting ./chunks/19chunk.wav
exporting ./chunks/20chunk.wav
exporting ./chunks/21chunk.wav
exporting ./chunks/22chunk.wav
exporting ./chunks/23chunk.wav
exporting ./chunks/24chunk.wav
exporting ./chunks/25chunk.wav
exporting ./chunks/26chunk.wav
exporting ./chunks/27chunk.wav


In [18]:
chunks_list

['./chunks/0chunk.wav',
 './chunks/1chunk.wav',
 './chunks/2chunk.wav',
 './chunks/3chunk.wav',
 './chunks/4chunk.wav',
 './chunks/5chunk.wav',
 './chunks/6chunk.wav',
 './chunks/7chunk.wav',
 './chunks/8chunk.wav',
 './chunks/9chunk.wav',
 './chunks/10chunk.wav',
 './chunks/11chunk.wav',
 './chunks/12chunk.wav',
 './chunks/13chunk.wav',
 './chunks/14chunk.wav',
 './chunks/15chunk.wav',
 './chunks/16chunk.wav',
 './chunks/17chunk.wav',
 './chunks/18chunk.wav',
 './chunks/19chunk.wav',
 './chunks/20chunk.wav',
 './chunks/21chunk.wav',
 './chunks/22chunk.wav',
 './chunks/23chunk.wav',
 './chunks/24chunk.wav',
 './chunks/25chunk.wav',
 './chunks/26chunk.wav',
 './chunks/27chunk.wav']

# Encoding the various chunks

In [23]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

In [21]:
sentences=[]
for x in chunks_list:    
    tsc, words = leopard.process_file(x)
    sentences.append(tsc)

In [24]:

sentence_embeddings = model.encode(sentences)

In [25]:
sentence_embeddings.shape

(28, 384)

In [26]:
query = ['algorithm problem']

query_embedding = model.encode(query).reshape(1, -1)
query_embedding[0].shape

(384,)

# Applying cosine similarity

In [27]:
import numpy as np
from numpy.linalg import norm

for i in range(len(sentences)):
    cosine = np.dot(sentence_embeddings[i],query_embedding[0])/(norm(sentence_embeddings[i])*norm(query_embedding[0]))
    print(cosine)

0.33584172
0.26900294
0.32595578
0.26785132
0.28343552
0.46457997
0.47252163
0.45274082
0.22316939
0.40343967
0.15847988
0.32861745
0.22016829
0.22356342
0.29657552
0.2722554
0.23051089
0.3096066
0.2104142
0.28147027
0.034395248
0.047312602
0.050770845
0.115280785
0.10401398
0.14482424
0.2371611
0.39728472


In [18]:
# Based on the cosine similarity, we can get the top 5 sentences. The last audio chunk correlates mostly to the given query by user.